In [ ]:
import cv2
import os
import numpy as np

# Charger le jeu de données
def load_dataset(dataset_paths):
    dataset = []
    for dataset_path in dataset_paths:
        for filename in os.listdir(dataset_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image = cv2.imread(os.path.join(dataset_path, filename))
                if image is not None:
                    dataset.append((os.path.join(dataset_path, filename), image))
    return dataset

# Extraire les caractéristiques avec SIFT
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

# Comparer les caractéristiques
def match_features(descriptor1, descriptor2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptor1, descriptor2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    return len(good_matches)

# Trouver les images les plus similaires
def find_similar_images(input_image_path, dataset_paths):
    input_image = cv2.imread(input_image_path)
    _, input_descriptors = extract_features(input_image)
    dataset = load_dataset(dataset_paths)

    scores = []
    for filepath, image in dataset:
        _, descriptors = extract_features(image)
        score = match_features(input_descriptors, descriptors)
        scores.append((filepath, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:3]  # Retourner les 3 meilleures correspondances

# Chemins des sous-répertoires
dataset_paths = [
    r"C:\Users\SONA8\Desktop\projet_eval\chat",
    r"C:\Users\SONA8\Desktop\projet_eval\lampe",
    r"C:\Users\SONA8\Desktop\projet_eval\train"
]

# Chemin de l'image d'entrée
input_image_path = r"C:\Users\SONA8\Desktop\projet_eval\chat\chat2.jpg"

# Exécuter la recherche
similar_images = find_similar_images(input_image_path, dataset_paths)
print("Les images les plus similaires sont :")
for filepath, score in similar_images:
    print(f"{filepath} avec un score de similarité de {score}")
